In [ ]:
import importlib
import math
import time

import numpy as np
import pandas as pd

import skimage
from skimage.io import imread
from skimage import color
from skimage.transform import rescale, resize, downscale_local_mean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d
from torch.optim import Adam
from torch.utils import data

import natural_images_dataset_loader
from natural_images_dataset_loader import get_dataset_info, get_files_label_map, make_loaders

In [ ]:
importlib.reload(natural_images_dataset_loader)
from natural_images_dataset_loader import get_class_label_map, NaturalImagesDataset

In [ ]:
%matplotlib inline

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
class NaturalImagesNet(nn.Module):
    
    def __init__(self, in_channels, classes_num):
        super(NaturalImagesNet, self).__init__()
        self.in_channels = in_channels
        self.conv0 = Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=3, stride=1, padding=0)
        self.fc_out = Linear(62 * 62 * in_channels, classes_num)
        
    def forward(self, x):
        #print("forward: x.shape before conv0(x)", x.shape)
        x = self.conv0(x)
        #print("forward: x.shape after conv0(x)", x.shape)
        x = x.view(-1, 62 * 62 * self.in_channels)
        x = self.fc_out(x)
        #x = F.log_softmax(x, dim=1)
        x = F.log_softmax(x)
        #print("forward, x after log_softmax:\n", x)
        return x

In [ ]:
params = {'batch_size': 4, 'shuffle': True}
class_label_map = get_class_label_map('../input/natural-images/natural_images/*')

ni_ds_train = NaturalImagesDataset(class_label_map, resize_sizes=(64, 64))
ni_train_dataloader = data.DataLoader(ni_ds_train, **params)

ni_ds_valid = NaturalImagesDataset(class_label_map, valid=True, resize_sizes=(64, 64))
ni_valid_loader = data.DataLoader(ni_ds_valid, **params)

In [ ]:
dataset_info = get_dataset_info('../input/natural-images/natural_images/*')
print("len(dataset_info):", len(dataset_info))
print("type(dataset_info[0][2]): ", type(dataset_info[0][2]))
print("dataset_info[0][2][:10]: ", dataset_info[0][2][:10])
files_label_map = get_files_label_map(dataset_info)
print(get_files_label_map(dataset_info)[2000:2020])
#print(type(make_loaders(files_label_map)))
train_dl, valid_dl, test_dl = make_loaders(files_label_map, train_test_split_ratio=0.25, train_valid_split_ratio=0.25)
print(type(train_dl), type(valid_dl), type(test_dl))

In [ ]:
classes_num = len(dataset_info)
print(classes_num)

In [ ]:
input_channels = 3

In [ ]:
learning_rate = 0.001

In [ ]:
#img, label = next(iter(ni_train_dataloader))
img, label = next(iter(train_dl))

In [ ]:
#type(img)
#img.shape

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print("device: ", device)

In [ ]:
model = NaturalImagesNet(input_channels, classes_num)
#model = NaturalImagesNet()

In [ ]:
model.to(device)

In [ ]:
criterion = CrossEntropyLoss()

In [ ]:
optimizer = Adam(model.parameters(), lr=learning_rate)

In [ ]:
num_of_epoch = 12

train_losses = []
val_losses = []

start_time = time.time()

for epoch in range(num_of_epoch):
    # loop over the dataset multiple times
    
    avg_train_cost = 0
    avg_val_cost = 0
    
    train_running_loss = 0.0
    val_running_loss = 0.0
    
    for i, data in enumerate(ni_train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        
        train_losses.append(loss)
        
        loss.backward()
        optimizer.step()

        # print statistics
        train_running_loss += loss.item()
        
        if i % 200 == 199:    # print every 2000 mini-batches
            print("epoch: {}, batch: {} train running loss: {}".format(epoch, i, train_running_loss / 20))
            train_running_loss = 0.0
            
        avg_train_cost = avg_train_cost + loss.data
                
    if epoch % 2 != 0:
        print("epoch {}, avg train cost: {}".format(epoch, avg_train_cost))
        avg_train_cost = 0
        
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(ni_valid_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)
                
            outputs = model(inputs)
                
            val_loss = criterion(outputs, labels)
            val_losses.append(val_loss)
                
            val_running_loss += val_loss.item()
                
            if i % 200 == 199:    # print every 2000 mini-batches
                print("epoch: {}, batch: {} val running loss: {}".format(epoch, i, val_running_loss / 20))
                val_running_loss = 0.0
                
        avg_val_cost = avg_val_cost + loss.data
                
        if epoch % 2 != 0:
            print("epoch {}, avg val cost: {}".format(epoch, avg_val_cost))
            avg_val_cost = 0



print('Finished Training')
time_elapsed = time.time() - start_time
print(
    'Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60
    )
)

In [ ]:
plt.plot(train_losses, label="Training loss")
plt.plot(val_losses, label="Validation loss")#x = self.relu(x)
plt.legend()
plt.show()

In [ ]:
#x, y = Variable(train_x), Variable(train_y, requires_grad=False)

#pred = model(x)

#final_pred = np.argmax(pred.data.numpy(), axis=1)

#accuracy_score(train_y, final_pred)

In [ ]:
PATH = './my_first_model.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
dataiter = iter(ni_valid_loader)
images, labels = dataiter.next()

In [ ]:
#print images
imshow(torchvision.utils.make_grid(images))
#print('GroundTruth: ', ' '.join('%5s' % class_label_map[labels[j]] for j in range(4)))

In [ ]:
#model0 = NaturalImagesNet()
#model0.load_state_dict(torch.load(PATH))

In [ ]:
images = images.to(device, dtype=torch.float32)

In [ ]:
#outputs = model0(images)
outputs = model(images)

In [ ]:
#_, predicted = torch.max(outputs, 1)

#print('Predicted: ', ' '.join('%5s' % class_label_map[predicted[j]]
#                              for j in range(4)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in ni_valid_loader:
        images, labels = data
        outputs = model(images.to(device, dtype=torch.float32))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in ni_valid_loader:
        images, labels = data
        labels = labels.to(device)
        outputs = model(images.to(device, dtype=torch.float32))
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        #for i in range(4):
        for i in range(2):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(8):
    print('Accuracy of %5s : %2d %%' % (
        class_label_map[i], 100 * class_correct[i] / class_total[i]))